In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

import os 
import glob
import json
from pathlib import Path
from src.annotation_utils import display_image_and_references
import numpy as np
import pandas as pd
import superintendent

### Load the labels.csv file 

In [ ]:
csv_file_path = "/home/jovyan/data/labels.csv"
labels_df = pd.read_csv(csv_file_path, index_col = ['lat_lon', 'timestamp'], dtype={'count': float})
len(labels_df)

### Get list of images that should be relabeled

In [ ]:
data_chips_dir = "/home/jovyan/data/chips/"

tobe_labeled_images = []
relabel_lat_lons = [[36.02, -5.36],[35.95, -5.61]] # list of locations we want to relabel

for loc in relabel_lat_lons:
    lat, lon = loc[0], loc[1]
    subdir = 'lat_{}_lon_{}'.format(str(lat).replace('.','_'), str(lon).replace('.','_'))       
    tobe_labeled_images.extend(Path(data_chips_dir, subdir).glob("img_ndwi*.png"))
print(f"{len(tobe_labeled_images)} images to be relabeled")    

### Load superintendent widget and labelling

In [ ]:
labeller = superintendent.ClassLabeller(
    features=tobe_labeled_images,
    options=[i for i in range(-1, 6)],
    display_func=display_image_and_references,
)

labeller

### Extract new labels and save them in labels_df

In [ ]:
for i in range(len(tobe_labeled_images)):
    timestamp = tobe_labeled_images[i].stem.split('t_')[1]
    lat_lon = tobe_labeled_images[i].parts[-2]
    count = labeller.new_labels[i]
    # overwrite if the 
    if count:
        labels_df.at[(lat_lon, timestamp)] = count

### Dump back to csv file

In [ ]:
labels_df.to_csv(csv_file_path)